In [12]:
import xarray as xr
import numpy as np
import os
from datetime import datetime

In [10]:
os.listdir('../reforecast_v12/slp/mean_hours')[0][-18:-8]

'2012021500'

In [2]:
era_mslp = xr.open_dataset('../era5/mslp.nc',chunks={'time':100,'latitude':100,'longitude':100})

In [3]:
era_mslp

,Array,Chunk
Bytes,20.05 GB,4.00 MB
Shape,"(14440, 241, 1440)","(100, 100, 100)"
Count,6526 Tasks,6525 Chunks
Type,float32,numpy.ndarray


In [23]:
files[0]

'nh_time_pres_msl_2012021500_mean.nc'

In [26]:
files = os.listdir('../reforecast_v12/slp/mean_hours')
file_dt = [datetime.strptime(n[-18:-8],'%Y%m%d%H') for n in files]
djf = [n for n in file_dt if n.month in [12,1,2] and n.year in np.arange(2007,2014.1,1)]
nov = [n for n in file_dt if n.month == 11 and n.year in np.arange(2007,2014.1,1)]
nov = [n for n in nov if n.day <= 23]
djfnov = djf + nov
files_ndjf = [n.strftime('../reforecast_v12/slp/mean_hours/nh_time_pres_msl_%Y%m%d%H_mean.nc') for n in djfnov]
# xr.open_dataset('../reforecast_v12/slp/mean_hours/nh_time_pres_msl_2019020100_mean.nc')

In [27]:
gefs_mslp = xr.open_mfdataset(files_ndjf,
                              coords='minimal',
                              data_vars='minimal',
                              compat='override',
                              combine='nested',concat_dim='date',chunks={'date':100,'latitude':100,'longitude':100,'time':1})

In [28]:
gefs_mslp

,Array,Chunk
Bytes,57.99 GB,40.00 kB
Shape,"(746, 56, 241, 1440)","(1, 1, 100, 100)"
Count,3760586 Tasks,1879920 Chunks
Type,float32,numpy.ndarray


In [6]:
gefs_mslp

,Array,Chunk
Bytes,195.90 GB,40.00 kB
Shape,"(2205, 64, 241, 1440)","(1, 1, 100, 100)"
Count,35817362 Tasks,6350400 Chunks
Type,float32,numpy.ndarray
